# **BERT Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Crime_Data_from_2020_to_Present (1).csv")
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829778 entries, 0 to 829777
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           829778 non-null  int64  
 1   Date Rptd       829778 non-null  object 
 2   DATE OCC        829778 non-null  object 
 3   TIME OCC        829778 non-null  int64  
 4   AREA            829778 non-null  int64  
 5   AREA NAME       829778 non-null  object 
 6   Rpt Dist No     829778 non-null  int64  
 7   Part 1-2        829778 non-null  int64  
 8   Crm Cd          829778 non-null  int64  
 9   Crm Cd Desc     829778 non-null  object 
 10  Mocodes         714922 non-null  object 
 11  Vict Age        829778 non-null  int64  
 12  Vict Sex        720479 non-null  object 
 13  Vict Descent    720471 non-null  object 
 14  Premis Cd       829768 non-null  float64
 15  Premis Desc     829286 non-null  object 
 16  Weapon Used Cd  289319 non-null  float64
 17  Weapon Des

In [ ]:
df = df.dropna(subset=['Vict Descent'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720471 entries, 0 to 829777
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           720471 non-null  int64  
 1   Date Rptd       720471 non-null  object 
 2   DATE OCC        720471 non-null  object 
 3   TIME OCC        720471 non-null  int64  
 4   AREA            720471 non-null  int64  
 5   AREA NAME       720471 non-null  object 
 6   Rpt Dist No     720471 non-null  int64  
 7   Part 1-2        720471 non-null  int64  
 8   Crm Cd          720471 non-null  int64  
 9   Crm Cd Desc     720471 non-null  object 
 10  Mocodes         714493 non-null  object 
 11  Vict Age        720471 non-null  int64  
 12  Vict Sex        720467 non-null  object 
 13  Vict Descent    720471 non-null  object 
 14  Premis Cd       720470 non-null  float64
 15  Premis Desc     719989 non-null  object 
 16  Weapon Used Cd  289138 non-null  float64
 17  Weapon Des

In [ ]:
object_cols = [i for i in df.columns if df[i].dtype == 'O']
num_cols = [i for i in df.columns if df[i].dtype != 'O']

print("All objective columns: ", object_cols)
print("All numeric columns: ", num_cols)

print("No. of unique values in each objective column")
df[object_cols].nunique()

All objective columns:  ['Date Rptd', 'DATE OCC', 'AREA NAME', 'Crm Cd Desc', 'Mocodes', 'Vict Sex', 'Vict Descent', 'Premis Desc', 'Weapon Desc', 'Status', 'Status Desc', 'LOCATION', 'Cross Street']
All numeric columns:  ['DR_NO', 'TIME OCC', 'AREA', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Vict Age', 'Premis Cd', 'Weapon Used Cd', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LAT', 'LON']
No. of unique values in each objective column


Date Rptd         1399
DATE OCC          1399
AREA NAME           21
Crm Cd Desc        137
Mocodes         276551
Vict Sex             5
Vict Descent        20
Premis Desc        306
Weapon Desc         79
Status               6
Status Desc          6
LOCATION         60995
Cross Street      8807
dtype: int64

In [ ]:
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720471 entries, 0 to 829777
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           720471 non-null  int64  
 1   TIME OCC        720471 non-null  int64  
 2   AREA            720471 non-null  int64  
 3   Rpt Dist No     720471 non-null  int64  
 4   Part 1-2        720471 non-null  int64  
 5   Crm Cd          720471 non-null  int64  
 6   Vict Age        720471 non-null  int64  
 7   Premis Cd       720470 non-null  float64
 8   Weapon Used Cd  289138 non-null  float64
 9   Crm Cd 1        720462 non-null  float64
 10  Crm Cd 2        60867 non-null   float64
 11  Crm Cd 3        2056 non-null    float64
 12  Crm Cd 4        61 non-null      float64
 13  LAT             720471 non-null  float64
 14  LON             720471 non-null  float64
dtypes: float64(8), int64(7)
memory usage: 87.9 MB


In [ ]:
num_cols.remove("Weapon Used Cd")
num_cols.remove("AREA")
num_cols.remove("Crm Cd 2")
num_cols.remove("Crm Cd 3")
num_cols.remove("Crm Cd 4")
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720471 entries, 0 to 829777
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DR_NO        720471 non-null  int64  
 1   TIME OCC     720471 non-null  int64  
 2   Rpt Dist No  720471 non-null  int64  
 3   Part 1-2     720471 non-null  int64  
 4   Crm Cd       720471 non-null  int64  
 5   Vict Age     720471 non-null  int64  
 6   Premis Cd    720470 non-null  float64
 7   Crm Cd 1     720462 non-null  float64
 8   LAT          720471 non-null  float64
 9   LON          720471 non-null  float64
dtypes: float64(4), int64(6)
memory usage: 60.5 MB


In [ ]:
cat_cols = ["AREA NAME", "Vict Sex", "Status"]

feat_cols = num_cols + cat_cols
label_col = "Vict Descent"

In [ ]:
sel_df = df[feat_cols].copy()

sel_df.head()

,DR_NO,TIME OCC,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,Crm Cd 1,LAT,LON,AREA NAME,Vict Sex,Status
0,10304468,2230,377,2,624,36,501.0,624.0,34.0141,-118.2978,Southwest,F,AO
1,190101086,330,163,2,624,25,102.0,624.0,34.0459,-118.2545,Central,M,IC
2,200110444,1200,155,2,845,0,726.0,845.0,34.0448,-118.2474,Central,X,AA
3,191501505,1730,1543,2,745,76,502.0,745.0,34.1685,-118.4019,N Hollywood,F,IC
4,191921269,415,1998,2,740,31,409.0,740.0,34.2198,-118.4468,Mission,X,IC


In [ ]:
sel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720471 entries, 0 to 829777
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DR_NO        720471 non-null  int64  
 1   TIME OCC     720471 non-null  int64  
 2   Rpt Dist No  720471 non-null  int64  
 3   Part 1-2     720471 non-null  int64  
 4   Crm Cd       720471 non-null  int64  
 5   Vict Age     720471 non-null  int64  
 6   Premis Cd    720470 non-null  float64
 7   Crm Cd 1     720462 non-null  float64
 8   LAT          720471 non-null  float64
 9   LON          720471 non-null  float64
 10  AREA NAME    720471 non-null  object 
 11  Vict Sex     720467 non-null  object 
 12  Status       720471 non-null  object 
dtypes: float64(4), int64(6), object(3)
memory usage: 77.0+ MB


In [ ]:
#filling all the feature column with mode value
sel_df = sel_df.apply(lambda col: col.fillna(col.mode()[0]))


sel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720471 entries, 0 to 829777
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DR_NO        720471 non-null  int64  
 1   TIME OCC     720471 non-null  int64  
 2   Rpt Dist No  720471 non-null  int64  
 3   Part 1-2     720471 non-null  int64  
 4   Crm Cd       720471 non-null  int64  
 5   Vict Age     720471 non-null  int64  
 6   Premis Cd    720471 non-null  float64
 7   Crm Cd 1     720471 non-null  float64
 8   LAT          720471 non-null  float64
 9   LON          720471 non-null  float64
 10  AREA NAME    720471 non-null  object 
 11  Vict Sex     720471 non-null  object 
 12  Status       720471 non-null  object 
dtypes: float64(4), int64(6), object(3)
memory usage: 77.0+ MB


In [ ]:
feature_df = pd.get_dummies(sel_df)
label_df = pd.get_dummies(df[label_col])

In [ ]:
feature_df.head()

,DR_NO,TIME OCC,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,Crm Cd 1,LAT,LON,...,Vict Sex_F,Vict Sex_H,Vict Sex_M,Vict Sex_X,Status_AA,Status_AO,Status_CC,Status_IC,Status_JA,Status_JO
0,10304468,2230,377,2,624,36,501.0,624.0,34.0141,-118.2978,...,1,0,0,0,0,1,0,0,0,0
1,190101086,330,163,2,624,25,102.0,624.0,34.0459,-118.2545,...,0,0,1,0,0,0,0,1,0,0
2,200110444,1200,155,2,845,0,726.0,845.0,34.0448,-118.2474,...,0,0,0,1,1,0,0,0,0,0
3,191501505,1730,1543,2,745,76,502.0,745.0,34.1685,-118.4019,...,1,0,0,0,0,0,0,1,0,0
4,191921269,415,1998,2,740,31,409.0,740.0,34.2198,-118.4468,...,0,0,0,1,0,0,0,1,0,0


In [ ]:
print(sel_df.columns)
print(feature_df.columns)

Index(['DR_NO', 'TIME OCC', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Vict Age',
       'Premis Cd', 'Crm Cd 1', 'LAT', 'LON', 'AREA NAME', 'Vict Sex',
       'Status'],
      dtype='object')
Index(['DR_NO', 'TIME OCC', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Vict Age',
       'Premis Cd', 'Crm Cd 1', 'LAT', 'LON', 'AREA NAME_77th Street',
       'AREA NAME_Central', 'AREA NAME_Devonshire', 'AREA NAME_Foothill',
       'AREA NAME_Harbor', 'AREA NAME_Hollenbeck', 'AREA NAME_Hollywood',
       'AREA NAME_Mission', 'AREA NAME_N Hollywood', 'AREA NAME_Newton',
       'AREA NAME_Northeast', 'AREA NAME_Olympic', 'AREA NAME_Pacific',
       'AREA NAME_Rampart', 'AREA NAME_Southeast', 'AREA NAME_Southwest',
       'AREA NAME_Topanga', 'AREA NAME_Van Nuys', 'AREA NAME_West LA',
       'AREA NAME_West Valley', 'AREA NAME_Wilshire', 'Vict Sex_-',
       'Vict Sex_F', 'Vict Sex_H', 'Vict Sex_M', 'Vict Sex_X', 'Status_AA',
       'Status_AO', 'Status_CC', 'Status_IC', 'Status_JA', 'Status_JO'],
      dty

In [ ]:
label_df.head()

,-,A,B,C,D,F,G,H,I,J,K,L,O,P,S,U,V,W,X,Z
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
print("Shape of input feature df: ", feature_df.shape)
print("Shape of output label df: ", label_df.shape)

Shape of input feature df:  (720471, 42)
Shape of output label df:  (720471, 20)


In [ ]:
# Assuming your data is loaded into feature_df and label_df
# feature_df has shape (1000, 42) and label_df has shape (100, 16)
n_feat = feature_df.shape[1]
n_class = label_df.shape[1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    feature_df.values, label_df.values, test_size=0.2, random_state=42
)

# Concatenating the input features into one **column**

In [ ]:
df["new_text"] = [" ".join([str(j) for j in df.iloc[i].values]) for i in range(len(df))]

final_df = df[["new_text", label_col]].copy()

# **Encoding the Target Feature**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

max_sequence_length = 144
# Encode labels
label_encoder = LabelEncoder()
final_df['label'] = label_encoder.fit_transform(final_df[label_col])

# Split the data into training and testing sets
train_df, test_df = train_test_split(final_df, test_size=0.2, random_state=42)

# **Pre-Trained BERT model**

In [ ]:
from transformers import TFBertModel, BertTokenizer
# Assuming you have loaded your BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizing input texts
X_train_sequences = [tokenizer.encode(text, add_special_tokens=True, max_length=max_sequence_length, truncation=True) for text in train_df['new_text']]
X_test_sequences = [tokenizer.encode(text, add_special_tokens=True, max_length=max_sequence_length, truncation=True) for text in test_df['new_text']]


# Pad sequences for consistent length
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

# Define input layer
num_classes = len(label_encoder.classes_)
input_layer = layers.Input(shape=(max_sequence_length,), dtype=tf.int32, name="input_layer")

# Load BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Freeze BERT layers
bert_model.trainable = False

# Get BERT output
bert_output = bert_model(input_layer)["last_hidden_state"]

# Global average pooling
pooled_output = layers.GlobalAveragePooling1D()(bert_output)

# Dropout layer
dropout_layer = layers.Dropout(0.2)(pooled_output)

# Output layer
output_layer = layers.Dense(num_classes, activation='softmax')(dropout_layer)

# Build and compile the model
bert_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
bert_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

bert_model.summary()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 144)]             0         
                                                                 
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPo   109482240 
 )                           olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 144, 768),                       
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                       
                                                             

# **Oversampling on the minority classes**

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

oversampler = RandomOverSampler(sampling_strategy='minority')
X_resampled, y_resampled = oversampler.fit_resample(X_train_padded, y_train)

In [ ]:
print(X_resampled.shape)
print(y_resampled.shape)

(779781, 144)
(779781, 20)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


epochs = 1
batch_size = 8

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# ModelCheckpoint callback to save the best model
checkpoint_filepath = 'best_model.h5'
model_checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', save_best_only=True)


bert_model.fit(X_resampled, y_resampled, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test), callbacks=[early_stopping,model_checkpoint])

# Evaluation

loss, accuracy = bert_model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%')

97473/97473 [==============================] - ETA: 0s - loss: 0.9486 - accuracy: 0.6578

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4503/4503 [==============================] - 1465s 325ms/step - loss: 0.9302 - accuracy: 0.7178
Test Loss: 0.9302, Test Accuracy: 71.78%


In [ ]:
# Make predictions on the test set
y_pred_prob = bert_model.predict(X_test_padded)


4503/4503 [==============================] - 1450s 321ms/step


In [ ]:

import numpy as np
from sklearn.metrics import classification_report

# Assuming y_pred_prob contains predicted probabilities for each class
y_pred_labels = np.argmax(y_pred_prob, axis=1)

y_true_labels = np.argmax(y_test, axis=1)

true_labels = label_encoder.inverse_transform(y_true_labels)
predicted_labels = label_encoder.inverse_transform(y_pred_labels)

# Print Classification Report
print(classification_report(true_labels, predicted_labels, zero_division=1))


              precision    recall  f1-score   support

           -       0.00      0.00      0.00         1
           A       0.92      0.17      0.29      3606
           B       0.74      0.69      0.71     23467
           C       1.00      0.00      0.00       644
           D       1.00      0.00      0.00        16
           F       1.00      0.00      0.00       714
           G       1.00      0.00      0.00         7
           H       0.67      0.93      0.78     51099
           I       1.00      0.00      0.00       160
           J       1.00      0.00      0.00       210
           K       1.00      0.00      0.00       902
           L       1.00      0.00      0.00        13
           O       0.98      0.08      0.15     13122
           P       1.00      0.00      0.00        41
           S       1.00      0.00      0.00        15
           U       1.00      0.00      0.00        36
           V       1.00      0.00      0.00       172
           W       0.71    